In [1]:
%cd /code

/code


In [10]:
import os

openai_token = os.getenv('OPENAI_TOKEN')


In [125]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(
    model=model_name,
    temperature=0,
    max_tokens=1000,
    openai_api_key=openai_token,
    verbose=True,
)

from langchain.callbacks import get_openai_callback
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
]

with get_openai_callback() as callback:
    response = llm(messages)

print(callback)
print(response.content)


Tokens Used: 33
	Prompt Tokens: 26
	Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $6.6e-05
The capital of France is Paris.


In [126]:
response

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)

In [44]:
print(f"{callback.prompt_tokens:,}")
print(f"{callback.completion_tokens:,}")
print(f"{callback.total_tokens:,}")
print(f"{callback.successful_requests:,}")
print(f"{callback.total_cost:.6f}")

26
7
33
1
0.000066


In [45]:
messages += [response]
messages += [HumanMessage(content="What is the lat/long?")]
messages

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}),
 HumanMessage(content='What is the capital of France?', additional_kwargs={}, example=False),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False),
 HumanMessage(content='What is the lat/long?', additional_kwargs={}, example=False)]

In [46]:
with get_openai_callback() as callback:
    response = llm(messages)

print(callback)
print(response.content)

Tokens Used: 75
	Prompt Tokens: 50
	Completion Tokens: 25
Successful Requests: 1
Total Cost (USD): $0.00015000000000000001
The latitude and longitude of Paris, France is 48.8566° N, 2.3522° E.


In [48]:
response.json()

'{"content": "The latitude and longitude of Paris, France is 48.8566\\u00b0 N, 2.3522\\u00b0 E.", "additional_kwargs": {}, "example": false}'

---

# Streaming

In [61]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import HumanMessage

from langchain.callbacks.openai_info import OpenAICallbackHandler
openai_callback = OpenAICallbackHandler()

model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), openai_callback],
    model=model_name,
    temperature=0,
    max_tokens=1000,
    openai_api_key=openai_token,
    verbose=True,
)

from langchain.callbacks import get_openai_callback
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
]


In [62]:
# https://github.com/hwchase17/langchain/issues/3114
with get_openai_callback() as callback_no_info:
    response = llm(messages)

The capital of France is Paris.

In [63]:
callback_no_info

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0

In [64]:
openai_callback

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0

In [57]:
print(callback)
print(response.content)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0
The capital of France is Paris.


how to implement streaming in streamlit https://github.com/hwchase17/chat-langchain/issues/39

how to implement Async callback

https://github.com/hwchase17/langchain/issues/4583

using streaming with FastAPI

- https://github.com/hwchase17/langchain/discussions/1706
- https://stackoverflow.com/questions/76357732/stream-a-langchain-openai-response-with-fastapi
- https://gist.github.com/ninely/88485b2e265d852d3feb8bd115065b1a?permalink_comment_id=4537719

---

In [69]:
import requests

url = 'http://127.0.0.1:8000/stream'
headers = {'Content-Type': 'application/json'}
data = {'message': 'hello!'}

response = requests.post(url, json=data, headers=headers)

print(response.status_code)
response

200


<Response [200]>

In [68]:
response.text

'data: \n\ndata: Hello\n\ndata:  there\n\ndata: !\n\ndata:  How\n\ndata:  can\n\ndata:  I\n\ndata:  assist\n\ndata:  you\n\ndata:  today\n\ndata: ?\n\ndata: \n\n'

In [86]:
import requests

url = 'http://127.0.0.1:8000/stream'
headers = {'Content-Type': 'application/json'}
data = {'message': 'Write a small poem no longer than 15 words.'}

response = requests.post(url, json=data, headers=headers, stream=True)

for chunk in response.iter_content(chunk_size=128):
    print(chunk.decode('utf-8'), end='')

Soft whispers in the breeze,
Leaves rustling with ease,
Birds chirping in the trees,
Nature's symphony, oh please!

The sun sets in the west,
Painting the sky with its best,
Colors of orange, red and gold,
A sight that never gets old.

Stars twinkle in the night,
A moon that shines so bright,
Peaceful and calm, oh what a delight,
This moment, oh so right.

Nature's beauty, a gift so pure,
A treasure that will forever endure,
Let's cherish and protect it for sure,
For generations to come, let it endure.

In [112]:
import requests

url = 'http://127.0.0.1:8000/stream'
headers = {'Content-Type': 'application/json'}
data = {'message': 'Write a small poem no longer than 15 words.'}

response = requests.post(url, json=data, headers=headers, stream=True)

if response.status_code == 200:
    # Extract the additional information from the response headers
    total_characters = response.headers.get("total_characters")

    # Process the streamed data
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            # Process the streamed data here
            print(chunk.decode(), end='')

    # Use the extracted additional information
    if total_characters:
        print()
        print("Additional Information:", total_characters)
else:
    print("Error:", response.status_code)


Sunrise hues paint the sky,
New day dawns, hope awakens high.

In [113]:
response.status_code

200

In [116]:
total_characters

In [117]:
response.headers

{'date': 'Mon, 05 Jun 2023 21:53:09 GMT', 'server': 'uvicorn', 'content-type': 'text/event-stream; charset=utf-8', 'Transfer-Encoding': 'chunked'}

---

# DuckDuckGo

In [119]:
# !pip install duckduckgo-search

In [123]:
import itertools
from duckduckgo_search import DDGS
ddgs = DDGS()
keywords = 'what is openai?'
ddgs_text_gen = ddgs.text(keywords, region='wt-wt', safesearch='Off', timelimit='y')
first_n_items = list(itertools.islice(ddgs_text_gen, 0, 5))
first_n_items

[{'title': 'About - OpenAI',
  'href': 'https://openai.com/about/',
  'body': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'title': 'OpenAI - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/OpenAI',
  'body': 'OpenAI is an American artificial intelligence (AI) research laboratory consisting of the non-profit OpenAI Incorporated and its for-profit subsidiary corporation OpenAI Limited Partnership.OpenAI conducts AI research with the declared intention of promoting and developing a friendly AI.. OpenAI was founded in 2015 by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej ...'},
 {'title': 'What is OpenAI? Definition and History from TechTarget',
  'href': 'https://www.

In [2]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper()
results = wrapper.results('what is openai?', 5)

for result in results:
    print(result["title"])
    print(result["link"])
    print(result["snippet"])
    print("---")


/usr/local/lib/python3.11/site-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/usr/local/lib/python3.11/site-packages/duckduckgo_search/compat.py:22: UserWarning: parameter time is deprecated, use parameter timelimit
  warnings.warn("parameter time is deprecated, use parameter timelimit")
/usr/local/lib/python3.11/site-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")
/usr/local/lib/python3.11/site-packages/duckduckgo_search/compat.py:26: UserWarning: parameter max_results is deprecated, use DDGS().text()
  warnings.warn("parameter max_results is deprecated, use DDGS().text()")


About - OpenAI
https://openai.com/about/
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
---
OpenAI - Wikipedia
https://en.wikipedia.org/wiki/OpenAI
OpenAI is an American artificial intelligence (AI) research laboratory consisting of the non-profit OpenAI Incorporated and its for-profit subsidiary corporation OpenAI Limited Partnership.OpenAI conducts AI research with the declared intention of promoting and developing friendly AI.. OpenAI was founded in 2015 by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy ...
---
What is OpenAI? Definition and History from TechTarget
https://www.techtarget.com/searchenterpriseai/definition/OpenAI
OpenAI. OpenAI is a private research lab

---

# Conversational Memory

https://www.pinecone.io/learn/langchain-conversational-memory/

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory



---

In [23]:
temp = {'context': 'adsf', 'more_context': 'afdafddafdsafsd'}
temp

{'context': 'adsf', 'more_context': 'afdafddafdsafsd'}

In [24]:
for k, v in temp.items():
    print(k, v)

context adsf
more_context afdafddafdsafsd


In [25]:
chat_gpt_message = """
        This is a prompt.

        It has fields like this {{context}}

        And this:

        ```
        {{more_context}}
        ```
        """
chat_gpt_message

'\n        This is a prompt.\n\n        It has fields like this {{context}}\n\n        And this:\n\n        ```\n        {{more_context}}\n        ```\n        '

In [26]:
for key, value in temp.items():
    chat_gpt_message = chat_gpt_message.replace("{{" + key + "}}", value)

chat_gpt_message

'\n        This is a prompt.\n\n        It has fields like this adsf\n\n        And this:\n\n        ```\n        afdafddafdsafsd\n        ```\n        '

In [27]:
None or ''

''

In [11]:
my_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]
#my_list = [1, 2, 3, 4, 5, 6, 7, 8]
# my_list = [1]
my_list = list(reversed(my_list))
print(my_list)
print(len(my_list))
for i in range(0, len(my_list) - 1, 2):
    item1 = my_list[i]
    item2 = my_list[i + 1]
    
    # Do something with item1 and item2
    print(item1, item2)

[8, 7, 6, 5, 4, 3, 2, 1, 0]
9
8 7
6 5
4 3
2 1


In [2]:
'asdf' in set(['sdfdsdf', 'ddd'])

False

In [2]:
import os

template_files = os.listdir('/code/prompt_templates/')
template_files

['comment-code.yml',
 'make-text-sound-better.yml',
 'write-unit-tests.yml',
 'python-doc-strings.yml',
 'improve-code.yml',
 'summarize-text.yml',
 'explain-code.yml',
 'find-issues-in-code.yml']

In [3]:
templates = {}
import yaml
for file_name in template_files:
    with open(os.path.join('/code/prompt_templates/', file_name)) as handle:
        yaml_data = yaml.safe_load(handle)
        template_name = yaml_data.pop('name')
        assert template_name not in templates
        templates[template_name] = yaml_data
templates

{'Comment code': {'category': 'code',
  'template': 'Create helpful comments throughout the following code:\n\n```\n{{code}}\n```\n'},
 'Make text sound better': {'category': 'text',
  'template': 'Improve the text below. Make it sound clear and concise. It should sound professional while remaining informal.\n\n```\n{{text}}\n```\n'},
 'Write tests for code': {'category': 'code',
  'template': 'Write unit tests using pytest for the following code. Try to test for both edge cases and standard cases.\n\n```\n{{code}}\n```\n'},
 'Create doc strings': {'category': 'code',
  'template': 'Create doc strings and type hints for the following python function. It should be in the format:\n\n```\ndef func(variable_a: str) -> int:\n    """\n    <This is the description>\n\n    Args:\n        variable_a: <description of variable_a>  \n    """\n    return 1\n```\n\nHere is the function:\n\n```\n{{python_function}}\n```\n'},
 'Improve and optimize code': {'category': 'code',
  'template': 'Improve an

In [4]:
print('`' + templates["Make text sound better"]['template'] + '`')

`Improve the text below. Make it sound clear and concise. It should sound professional while remaining informal.

```
{{text}}
```
`


In [5]:
list(templates.keys())

['Comment code',
 'Make text sound better',
 'Write tests for code',
 'Create doc strings',
 'Improve and optimize code',
 'Summarize text',
 'Explain code',
 'Find issues in code']

In [10]:
templates['Comment code']['category']

'code'

In [12]:
sorted_dict = sorted(templates.items(), key=lambda x: (x[1]['category'], x[1]['template']))
sorted_dict

[('Create doc strings',
  {'category': 'code',
   'template': 'Create doc strings and type hints for the following python function. It should be in the format:\n\n```\ndef func(variable_a: str) -> int:\n    """\n    <This is the description>\n\n    Args:\n        variable_a: <description of variable_a>  \n    """\n    return 1\n```\n\nHere is the function:\n\n```\n{{python_function}}\n```\n'}),
 ('Comment code',
  {'category': 'code',
   'template': 'Create helpful comments throughout the following code:\n\n```\n{{code}}\n```\n'}),
 ('Explain code',
  {'category': 'code',
   'template': 'Explain the following code:\n\n```\n{{code}}\n```\n'}),
 ('Find issues in code',
  {'category': 'code',
   'template': 'Find issues in the following code:\n\n```\n{{code}}\n```\n'}),
 ('Improve and optimize code',
  {'category': 'code',
   'template': 'Improve and optimize the following code. Explain the the improvements made.\n\n```\n{{code}}\n```\n'}),
 ('Write tests for code',
  {'category': 'code',